In [0]:
#importing packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
import builtins
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType
from pyspark.sql.functions import to_date, to_timestamp, col

**Campaign Table**

In [0]:
def generate_campaign_results_table(n_campaigns=156):
    """
    Generate sample data for Unity Catalog table: campaign_results
    
    This table contains:
    - Campaign ID (unique identifier)  
    - Total spend per campaign
    - Test sales (treatment group with marketing)
    - Control sales (holdout group without marketing)
    - Campaign metadata
    
    Args:
        n_campaigns: Number of campaigns to generate (default: 156 = 3 years weekly)
    
    Returns:
        DataFrame ready for Unity Catalog table
    """
    np.random.seed(42)
    
    # Generate campaign IDs
    campaign_ids = [f"CAMP_{str(i+1).zfill(4)}" for i in range(n_campaigns)]
    
    # Generate time periods (weekly campaigns over 3 years)
    start_date = datetime(2021, 1, 4)  # Start on Monday
    campaign_dates = [start_date + timedelta(weeks=i) for i in range(n_campaigns)]
    
    # Generate realistic campaign spends (vary by season and strategy)
    base_spend = 50000  # Average weekly spend
    seasonal_multipliers = []
    
    for i, date in enumerate(campaign_dates):
        # Seasonal effects
        month = date.month
        if month in [11, 12]:  # Holiday season
            seasonal_mult = 1.8
        elif month in [6, 7, 8]:  # Summer campaigns  
            seasonal_mult = 1.3
        elif month in [1, 9]:  # New Year, Back to School
            seasonal_mult = 1.4
        else:
            seasonal_mult = 1.0
            
        # Add some random variation
        random_mult = np.random.uniform(0.7, 1.4)
        seasonal_multipliers.append(seasonal_mult * random_mult)
    
    total_spends = [base_spend * mult for mult in seasonal_multipliers]
    
    # Generate control sales (baseline business performance)
    # This represents what sales would be WITHOUT marketing
    base_control_sales = 75000  # Baseline weekly sales
    control_sales = []
    
    for i, date in enumerate(campaign_dates):
        # Trend component (business growing 15% per year)
        trend_mult = 1 + (i / 52) * 0.15
        
        # Seasonal component for control sales
        month = date.month
        if month in [11, 12]:  # Holiday shopping
            seasonal_control = 1.6
        elif month in [6, 7]:  # Summer activity
            seasonal_control = 1.2  
        elif month in [1]:  # January dip
            seasonal_control = 0.8
        else:
            seasonal_control = 1.0
            
        # Random variation
        noise = np.random.normal(1, 0.1)
        
        control = base_control_sales * trend_mult * seasonal_control * noise
        control_sales.append(max(control, 10000))  # Floor at 10K
    
    # Generate test sales (control + marketing lift)
    test_sales = []
    
    for i in range(n_campaigns):
        # Marketing lift depends on spend efficiency
        spend = total_spends[i]
        control = control_sales[i]
        
        # Diminishing returns: efficiency decreases with higher spend
        spend_efficiency = 0.8 * (spend / 100000) ** -0.3  # Power law
        
        # Base lift from marketing (varies by period)
        base_lift_rate = np.random.uniform(0.15, 0.35)  # 15-35% lift
        
        # Calculate marketing lift
        marketing_lift = spend * spend_efficiency * base_lift_rate / 10000
        
        # Add some noise to test sales
        noise = np.random.normal(1, 0.08)
        
        test = control + marketing_lift * noise
        test_sales.append(test)
    
    # Generate campaign metadata
    campaign_types = np.random.choice(['Brand', 'Performance', 'Hybrid'], 
                                     size=n_campaigns, 
                                     p=[0.3, 0.4, 0.3])
    regions = np.random.choice(['North', 'South', 'East', 'West', 'National'], 
                              size=n_campaigns, 
                              p=[0.15, 0.15, 0.15, 0.15, 0.40])
    
    # Create the campaign results table
    campaign_results_df = pd.DataFrame({
        'campaign_id': campaign_ids,
        'campaign_week': [d.strftime('%Y-%W') for d in campaign_dates],
        'campaign_start_date': [d.strftime('%Y-%m-%d') for d in campaign_dates],  # Convert to string
        'campaign_type': campaign_types,
        'region': regions,
        'total_spend': [float(round(spend, 2)) for spend in total_spends],  # Ensure float
        'test_sales': [float(round(sales, 2)) for sales in test_sales],  # Ensure float
        'control_sales': [float(round(sales, 2)) for sales in control_sales],  # Ensure float
        'created_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # Convert to string
        'data_source': 'marketing_experiments'
    })
    
    # Add calculated fields - ensure all are proper numeric types
    campaign_results_df['incremental_sales'] = (campaign_results_df['test_sales'] - campaign_results_df['control_sales']).astype(float)
    campaign_results_df['lift_percent'] = ((campaign_results_df['incremental_sales'] / campaign_results_df['control_sales']) * 100).round(2).astype(float)
    campaign_results_df['roas'] = (campaign_results_df['test_sales'] / campaign_results_df['total_spend']).round(3).astype(float)
    campaign_results_df['iroas'] = (campaign_results_df['incremental_sales'] / campaign_results_df['total_spend']).round(3).astype(float)
    
    return campaign_results_df

**Campaign Tactics Table**

In [0]:


def generate_campaign_tactics_table(campaign_results_df):
    """
    Generate sample data for Unity Catalog table: campaign_tactics
    
    This table contains:
    - Campaign ID (foreign key to campaign_results)
    - Tactic breakdown (video, audio, display, search, social, etc.)
    - Spend allocation per tactic
    
    Args:
        campaign_results_df: DataFrame from generate_campaign_results_table()
        
    Returns:
        DataFrame ready for Unity Catalog table
    """
    np.random.seed(42)
    
    # Define available tactics and their typical characteristics
    tactics_info = {
        'video': {'min_pct': 0.05, 'max_pct': 0.45, 'weight': 0.25},
        'audio': {'min_pct': 0.05, 'max_pct': 0.35, 'weight': 0.15},  
        'display': {'min_pct': 0.10, 'max_pct': 0.40, 'weight': 0.20},
        'search': {'min_pct': 0.15, 'max_pct': 0.50, 'weight': 0.25},
        'social': {'min_pct': 0.05, 'max_pct': 0.30, 'weight': 0.15},
        'connected_tv': {'min_pct': 0.00, 'max_pct': 0.25, 'weight': 0.10},
        'podcast': {'min_pct': 0.00, 'max_pct': 0.15, 'weight': 0.05},
        'influencer': {'min_pct': 0.00, 'max_pct': 0.20, 'weight': 0.08},
        'email': {'min_pct': 0.02, 'max_pct': 0.10, 'weight': 0.05},
        'direct_mail': {'min_pct': 0.00, 'max_pct': 0.15, 'weight': 0.03}
    }
    
    tactics_data = []
    
    for _, campaign in campaign_results_df.iterrows():
        campaign_id = campaign['campaign_id']
        total_spend = campaign['total_spend']
        campaign_type = campaign['campaign_type']
        
        # Adjust tactic mix based on campaign type
        if campaign_type == 'Brand':
            # Brand campaigns favor video, audio, social
            tactics_weights = {'video': 0.35, 'audio': 0.20, 'display': 0.15, 'search': 0.10, 
                             'social': 0.20, 'connected_tv': 0.15, 'podcast': 0.10, 'influencer': 0.12,
                             'email': 0.03, 'direct_mail': 0.05}
        elif campaign_type == 'Performance':
            # Performance campaigns favor search, display, social
            tactics_weights = {'video': 0.15, 'audio': 0.08, 'display': 0.30, 'search': 0.40,
                             'social': 0.25, 'connected_tv': 0.05, 'podcast': 0.02, 'influencer': 0.05,
                             'email': 0.08, 'direct_mail': 0.02}
        else:  # Hybrid
            # Balanced approach
            tactics_weights = {'video': 0.25, 'audio': 0.15, 'display': 0.25, 'search': 0.25,
                             'social': 0.18, 'connected_tv': 0.10, 'podcast': 0.06, 'influencer': 0.08,
                             'email': 0.05, 'direct_mail': 0.03}
        
        # Generate random allocation but weighted by campaign type
        selected_tactics = []
        
        # Always include core tactics
        core_tactics = ['video', 'audio', 'display', 'search']
        for tactic in core_tactics:
            if np.random.random() < 0.9:  # 90% chance to include core tactics
                selected_tactics.append(tactic)
        
        # Randomly add other tactics
        other_tactics = ['social', 'connected_tv', 'podcast', 'influencer', 'email', 'direct_mail'] 
        for tactic in other_tactics:
            if np.random.random() < tactics_weights.get(tactic, 0.1):
                selected_tactics.append(tactic)
        
        # Ensure we have at least 3 tactics
        if len(selected_tactics) < 3:
            remaining = [t for t in tactics_info.keys() if t not in selected_tactics]
            remaining = list(remaining)  # Convert to list for numpy
            need = min(3 - len(selected_tactics), len(remaining))
            if need > 0 and len(remaining) > 0:
                additional = np.random.choice(remaining, size=need, replace=False)
                selected_tactics.extend(additional)
        
        # Generate spend allocation
        # Use Dirichlet distribution for realistic allocation
        alpha_values = [tactics_weights.get(tactic, 0.1) * 10 for tactic in selected_tactics]
        
        # Ensure we have valid alpha values
        alpha_values = [max(val, 0.1) for val in alpha_values]  # Minimum alpha of 0.1
        
        try:
            allocations = np.random.dirichlet(alpha_values)
        except ValueError:
            # Fallback to simple random allocation if Dirichlet fails
            allocations = np.random.random(len(selected_tactics))
            allocations = allocations / allocations.sum()  # Normalize to sum to 1
        
        # Create records for each tactic
        for i, tactic in enumerate(selected_tactics):
            spend_amount = total_spend * allocations[i]
            
            tactics_data.append({
                'campaign_id': campaign_id,
                'tactic': tactic,
                'spend_amount': round(spend_amount, 2),
                'spend_percentage': round(allocations[i] * 100, 1),
                'tactic_category': get_tactic_category(tactic),
                'channel_type': get_channel_type(tactic),
                'created_timestamp': datetime.now(),
                'data_source': 'media_planning'
            })
    
    return pd.DataFrame(tactics_data)


In [0]:
def get_tactic_category(tactic):
    """Helper function to categorize tactics"""
    categories = {
        'video': 'Video',
        'audio': 'Audio', 
        'display': 'Display',
        'search': 'Search',
        'social': 'Social',
        'connected_tv': 'Video',
        'podcast': 'Audio',
        'influencer': 'Social',
        'email': 'Direct',
        'direct_mail': 'Direct'
    }
    return categories.get(tactic, 'Other')

def get_channel_type(tactic):
    """Helper function to classify channel types"""
    digital_tactics = ['video', 'display', 'search', 'social', 'connected_tv', 'podcast', 'email']
    return 'Digital' if tactic in digital_tactics else 'Traditional'

In [0]:
def create_unity_catalog_ddl():
    """
    Generate DDL statements for Unity Catalog tables
    """
    ddl_statements = """
-- =====================================================
-- Unity Catalog DDL Statements for MMM Sample Data
-- =====================================================

-- Table 1: Campaign Results (Test vs Control)
CREATE OR REPLACE TABLE tadityadb.default.campaign_results (
  campaign_id STRING NOT NULL COMMENT 'Unique campaign identifier',
  campaign_week STRING COMMENT 'Campaign week in YYYY-WW format',
  campaign_start_date DATE COMMENT 'Campaign start date',
  campaign_type STRING COMMENT 'Brand, Performance, or Hybrid campaign',
  region STRING COMMENT 'Geographic region for campaign',
  total_spend DECIMAL(12,2) COMMENT 'Total campaign spend in USD',
  test_sales DECIMAL(12,2) COMMENT 'Sales from treatment group (with marketing)',
  control_sales DECIMAL(12,2) COMMENT 'Sales from control group (without marketing)',
  incremental_sales DECIMAL(12,2) COMMENT 'Test sales - Control sales', 
  lift_percent DECIMAL(5,2) COMMENT 'Percentage lift vs control group',
  roas DECIMAL(8,3) COMMENT 'Return on Ad Spend (test_sales / total_spend)',
  iroas DECIMAL(8,3) COMMENT 'Incremental ROAS (incremental_sales / total_spend)',
  created_timestamp TIMESTAMP COMMENT 'Record creation timestamp',
  data_source STRING COMMENT 'Source system for this data'
) 
USING DELTA 
PARTITIONED BY (campaign_week)
COMMENT 'Campaign performance data with test/control results for MMM analysis';

-- Table 2: Campaign Tactics (Spend Breakdown)  
CREATE OR REPLACE TABLE tadityadb.default.campaign_tactics (
  campaign_id STRING NOT NULL COMMENT 'Foreign key to campaign_results table',
  tactic STRING NOT NULL COMMENT 'Marketing tactic (video, audio, display, etc.)',
  spend_amount DECIMAL(12,2) COMMENT 'Amount spent on this tactic in USD',
  spend_percentage DECIMAL(5,1) COMMENT 'Percentage of total campaign spend',
  tactic_category STRING COMMENT 'High-level category (Video, Audio, Display, etc.)',
  channel_type STRING COMMENT 'Digital or Traditional channel type',
  created_timestamp TIMESTAMP COMMENT 'Record creation timestamp',
  data_source STRING COMMENT 'Source system for this data'
)
USING DELTA
COMMENT 'Detailed spend breakdown by marketing tactic for each campaign';

-- Create indexes for better query performance
--CREATE INDEX idx_campaign_results_date ON your_catalog.mmm_schema.--campaign_results (campaign_start_date);
--CREATE INDEX idx_campaign_tactics_id ON your_catalog.mmm_schema.--campaign_tactics (campaign_id);

-- =====================================================
-- Sample Queries for MMM Analysis
-- =====================================================

-- Query 1: Basic campaign performance summary
SELECT 
  campaign_week,
  COUNT(*) as num_campaigns,
  SUM(total_spend) as total_spend,
  SUM(incremental_sales) as total_incremental_sales,
  AVG(iroas) as avg_iroas,
  AVG(lift_percent) as avg_lift_percent
FROM tadityadb.default.campaign_results 
GROUP BY campaign_week
ORDER BY campaign_week;

-- Query 2: Tactic performance analysis
SELECT 
  t.tactic,
  COUNT(DISTINCT t.campaign_id) as num_campaigns,
  SUM(t.spend_amount) as total_spend,
  SUM(c.incremental_sales) as total_incremental_sales,
  SUM(c.incremental_sales) / SUM(t.spend_amount) as avg_iroas
FROM tadityadb.default.campaign_tactics t
JOIN tadityadb.default.campaign_results c ON t.campaign_id = c.campaign_id
GROUP BY t.tactic
ORDER BY avg_iroas DESC;

-- Query 3: Weekly data for MMM model (pivot tactics to columns)
WITH weekly_tactics AS (
  SELECT 
    c.campaign_week,
    c.campaign_start_date,
    c.incremental_sales,
    c.total_spend,
    t.tactic,
    t.spend_amount
  FROM tadityadb.default.campaign_results c
  JOIN tadityadb.default.campaign_tactics t ON c.campaign_id = t.campaign_id
)
SELECT 
  campaign_week,
  campaign_start_date,
  SUM(incremental_sales) as response_variable,
  SUM(CASE WHEN tactic = 'video' THEN spend_amount ELSE 0 END) as video_spend,
  SUM(CASE WHEN tactic = 'audio' THEN spend_amount ELSE 0 END) as audio_spend,
  SUM(CASE WHEN tactic = 'display' THEN spend_amount ELSE 0 END) as display_spend,
  SUM(CASE WHEN tactic = 'search' THEN spend_amount ELSE 0 END) as search_spend,
  SUM(CASE WHEN tactic = 'social' THEN spend_amount ELSE 0 END) as social_spend,
  SUM(total_spend) as total_spend
FROM weekly_tactics
GROUP BY campaign_week, campaign_start_date
ORDER BY campaign_start_date;
"""
    return ddl_statements

In [0]:
def create_databricks_loader_notebook():
    """
    Generate Databricks notebook code for loading the sample data
    """
    notebook_code = '''
# Databricks notebook source
# =====================================================  
# MMM Sample Data Loader for Unity Catalog
# =====================================================

# COMMAND ----------

# MAGIC %md
# MAGIC ## Marketing Mix Model - Sample Data Generator
# MAGIC 
# MAGIC This notebook generates realistic sample data for MMM analysis:
# MAGIC - **campaign_results**: Test vs Control sales data  
# MAGIC - **campaign_tactics**: Spend breakdown by marketing tactic
# MAGIC 
# MAGIC The data includes realistic patterns:
# MAGIC - Seasonal effects (holidays, summer)
# MAGIC - Diminishing returns in marketing effectiveness
# MAGIC - Various marketing tactics (video, audio, display, search, etc.)
# MAGIC - Test/control experimental design

# COMMAND ----------

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Initialize Spark session
spark = SparkSession.builder.appName("MMM_Data_Generator").getOrCreate()

# COMMAND ----------

# MAGIC %md
# MAGIC ### Step 1: Generate Campaign Results Data

# COMMAND ----------

# [Insert the generate_campaign_results_table function here]
# [Insert the generate_campaign_tactics_table function here]  
# [Insert helper functions here]

# Generate the sample data
print("Generating campaign results data...")
campaign_results_df = generate_campaign_results_table(156)  # 3 years of weekly data

print("Generating campaign tactics data...")
campaign_tactics_df = generate_campaign_tactics_table(campaign_results_df)

print(f"Generated {len(campaign_results_df)} campaigns")
print(f"Generated {len(campaign_tactics_df)} tactic records")

# COMMAND ----------

# MAGIC %md
# MAGIC ### Step 2: Data Quality Checks

# COMMAND ----------

# Display sample data
print("CAMPAIGN RESULTS - First 5 rows:")
display(campaign_results_df.head())

print("\\nCAMPAIGN TACTICS - First 10 rows:")
display(campaign_tactics_df.head(10))

# Summary statistics
print("\\nCAMPAIGN RESULTS SUMMARY:")
display(campaign_results_df.describe())

print("\\nTACTICS SPEND SUMMARY:")
tactics_summary = campaign_tactics_df.groupby('tactic')['spend_amount'].agg(['count', 'sum', 'mean']).round(2)
display(tactics_summary)

# COMMAND ----------

# MAGIC %md  
# MAGIC ### Step 3: Load Data to Unity Catalog

# COMMAND ----------

# Convert to Spark DataFrames
spark_campaign_results = spark.createDataFrame(campaign_results_df)
spark_campaign_tactics = spark.createDataFrame(campaign_tactics_df)

# Define your catalog and schema names
CATALOG_NAME = "your_catalog"  # Change this to your catalog
SCHEMA_NAME = "mmm_schema"     # Change this to your schema

# Write to Unity Catalog tables
print("Writing campaign_results to Unity Catalog...")
(spark_campaign_results
 .write
 .mode("overwrite")
 .option("mergeSchema", "true")  
 .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.campaign_results"))

print("Writing campaign_tactics to Unity Catalog...")
(spark_campaign_tactics
 .write
 .mode("overwrite") 
 .option("mergeSchema", "true")
 .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.campaign_tactics"))

print("✅ Data successfully loaded to Unity Catalog!")

# COMMAND ----------

# MAGIC %md
# MAGIC ### Step 4: Validate Data in Unity Catalog

# COMMAND ----------

# Query the tables to validate
print("Validating campaign_results table:")
results_count = spark.sql(f"SELECT COUNT(*) as count FROM {CATALOG_NAME}.{SCHEMA_NAME}.campaign_results").collect()[0]['count']
print(f"Total campaigns: {results_count}")

print("\\nValidating campaign_tactics table:")  
tactics_count = spark.sql(f"SELECT COUNT(*) as count FROM {CATALOG_NAME}.{SCHEMA_NAME}.campaign_tactics").collect()[0]['count']
print(f"Total tactic records: {tactics_count}")

print("\\nTactic breakdown:")
tactic_breakdown = spark.sql(f"""
SELECT tactic, COUNT(*) as campaigns, ROUND(SUM(spend_amount), 2) as total_spend
FROM {CATALOG_NAME}.{SCHEMA_NAME}.campaign_tactics 
GROUP BY tactic 
ORDER BY total_spend DESC
""")
display(tactic_breakdown)

# COMMAND ----------

# MAGIC %md
# MAGIC ### Step 5: Create MMM Analysis View

# COMMAND ----------

# Create a view that pivots tactics for MMM analysis
mmm_view_sql = f"""
CREATE OR REPLACE VIEW {CATALOG_NAME}.{SCHEMA_NAME}.mmm_weekly_data AS
WITH weekly_tactics AS (
  SELECT 
    c.campaign_week,
    c.campaign_start_date,
    c.incremental_sales as response_variable,
    c.total_spend,
    c.lift_percent,
    c.iroas,
    t.tactic,
    t.spend_amount
  FROM {CATALOG_NAME}.{SCHEMA_NAME}.campaign_results c
  JOIN {CATALOG_NAME}.{SCHEMA_NAME}.campaign_tactics t ON c.campaign_id = t.campaign_id
)
SELECT 
  campaign_week,
  campaign_start_date,
  SUM(response_variable) as incremental_sales,
  SUM(CASE WHEN tactic = 'video' THEN spend_amount ELSE 0 END) as video_spend,
  SUM(CASE WHEN tactic = 'audio' THEN spend_amount ELSE 0 END) as audio_spend,
  SUM(CASE WHEN tactic = 'display' THEN spend_amount ELSE 0 END) as display_spend,
  SUM(CASE WHEN tactic = 'search' THEN spend_amount ELSE 0 END) as search_spend,
  SUM(CASE WHEN tactic = 'social' THEN spend_amount ELSE 0 END) as social_spend,
  SUM(CASE WHEN tactic = 'connected_tv' THEN spend_amount ELSE 0 END) as connected_tv_spend,
  SUM(total_spend) as total_spend
FROM weekly_tactics
GROUP BY campaign_week, campaign_start_date
ORDER BY campaign_start_date
"""

spark.sql(mmm_view_sql)
print("✅ MMM analysis view created!")

# Display the view
print("\\nMMM Weekly Data (first 10 rows):")
display(spark.sql(f"SELECT * FROM {CATALOG_NAME}.{SCHEMA_NAME}.mmm_weekly_data LIMIT 10"))

# COMMAND ----------

# MAGIC %md
# MAGIC ### Ready for MMM Analysis! 🚀
# MAGIC 
# MAGIC Your data is now ready in Unity Catalog. Use this query to pull data for MMM:
# MAGIC 
# MAGIC ```sql
# MAGIC SELECT * FROM your_catalog.mmm_schema.mmm_weekly_data
# MAGIC ORDER BY campaign_start_date
# MAGIC ```
'''
    return notebook_code

In [0]:
# Example usage
if __name__ == "__main__":
    print("Marketing Mix Model - Unity Catalog Sample Data Generator")
    print("=" * 65)
    
    print("\\n1. Generating Campaign Results Data...")
    campaign_results = generate_campaign_results_table(156)  # 3 years weekly
    
    print("\\n2. Generating Campaign Tactics Data...")
    campaign_tactics = generate_campaign_tactics_table(campaign_results)
    
    print(f"\\n📊 DATA SUMMARY:")
    print(f"   • Total Campaigns: {len(campaign_results)}")
    print(f"   • Total Tactic Records: {len(campaign_tactics)}")
    print(f"   • Date Range: {campaign_results['campaign_start_date'].min()} to {campaign_results['campaign_start_date'].max()}")
    print(f"   • Total Spend: ${campaign_results['total_spend'].sum():,.2f}")
    print(f"   • Average iROAS: {campaign_results['iroas'].mean():.2f}")
    
    print(f"\\n📈 CAMPAIGN RESULTS SAMPLE:")
    print(campaign_results[['campaign_id', 'campaign_week', 'total_spend', 'test_sales', 'control_sales', 'incremental_sales', 'iroas']].head(10))
    
    print(f"\\n📊 TACTICS BREAKDOWN:")
    tactics_summary = campaign_tactics.groupby('tactic').agg({
        'spend_amount': ['count', 'sum', 'mean']
    }).round(2)
    tactics_summary.columns = ['Campaigns', 'Total_Spend', 'Avg_Spend']
    print(tactics_summary.sort_values('Total_Spend', ascending=False))
    
    print(f"\\n🗄️  UNITY CATALOG DDL:")
    print("Use this DDL to create your tables:")
    ddl = create_unity_catalog_ddl()
    print(ddl)
    
    print(f"\\n💻 DATABRICKS NOTEBOOK:")  
    print("Copy this code into a Databricks notebook to load your data:")
    notebook = create_databricks_loader_notebook()
    
    # Save files for easy access
    campaign_results.to_csv('campaign_results_sample.csv', index=False)
    campaign_tactics.to_csv('campaign_tactics_sample.csv', index=False)
    
    with open('unity_catalog_ddl.sql', 'w') as f:
        f.write(ddl)
        
    with open('databricks_loader_notebook.py', 'w') as f:
        f.write(notebook)
    
    print(f"\\n✅ FILES CREATED:")
    print(f"   • campaign_results_sample.csv")  
    print(f"   • campaign_tactics_sample.csv")
    print(f"   • unity_catalog_ddl.sql")
    print(f"   • databricks_loader_notebook.py")
    
    print(f"\\n🚀 NEXT STEPS:")
    print(f"   1. Update catalog/schema names in the DDL")
    print(f"   2. Run the DDL in Databricks to create tables") 
    print(f"   3. Use the notebook code to load sample data")
    print(f"   4. Query: SELECT * FROM your_catalog.mmm_schema.mmm_weekly_data")
    print(f"   5. Calculate response = incremental_sales and run your MMM!")
